In [109]:
from pyspark.sql import SparkSession
from pyspark.ml import pipeline
from pyspark.sql import functions as F
from pyspark.sql.types import NumericType


In [110]:
# Create spark session 
spark = SparkSession.builder.appName('PreprocessPipeline').getOrCreate()


In [111]:
# read file csv 
data = spark.read.csv('../data/vietnam_housing_dataset.csv', header= True, inferSchema= True)

In [112]:
data.show(10)

+--------------------+----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+
|             Address|Area|Frontage|Access Road|House direction|Balcony direction|Floors|Bedrooms|Bathrooms|    Legal status|Furniture state|Price|
+--------------------+----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+
|Dự án The Empire ...|84.0|    NULL|       NULL|           NULL|             NULL|     4|    NULL|     NULL|Have certificate|           NULL|  8.6|
|Dự án The Crown -...|60.0|    NULL|       NULL|           NULL|             NULL|     5|    NULL|     NULL|            NULL|           NULL|  7.5|
|Dự án The Crown -...|90.0|     6.0|       13.0|     Đông - Bắc|       Đông - Bắc|     5|    NULL|     NULL|   Sale contract|           NULL|  8.9|
|Đường Nguyễn Văn ...|54.0|    NULL|        3.5|      Tây - Nam|        Tây - Nam|     2|       2|        3|Have

In [113]:
# kiểm tra kiểu dữ liệu của  features
data.dtypes

[('Address', 'string'),
 ('Area', 'double'),
 ('Frontage', 'double'),
 ('Access Road', 'double'),
 ('House direction', 'string'),
 ('Balcony direction', 'string'),
 ('Floors', 'int'),
 ('Bedrooms', 'int'),
 ('Bathrooms', 'int'),
 ('Legal status', 'string'),
 ('Furniture state', 'string'),
 ('Price', 'double')]

In [114]:
# số lượng giá trị null trong từ cột
null_count_df = data.select( [ F.count( F.when(F.col(col=c).isNull(), c)).alias(c) for c in data.columns] )
null_count_df.show()


+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+
|Address|Area|Frontage|Access Road|House direction|Balcony direction|Floors|Bedrooms|Bathrooms|Legal status|Furniture state|Price|
+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+
|      0|   0|   11563|      13296|          21238|            24982|  3603|    5161|     7073|        4506|          14118|    0|
+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+



In [115]:
print("Số bản ghi của tập dữ liệu:" , data.count() )

Số bản ghi của tập dữ liệu: 30228


In [116]:
# đổi tên các cột
for col in data.columns:
    data = data.withColumnRenamed(col, col.replace(' ','_'))
data.show(5)

+--------------------+----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+
|             Address|Area|Frontage|Access_Road|House_direction|Balcony_direction|Floors|Bedrooms|Bathrooms|    Legal_status|Furniture_state|Price|
+--------------------+----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+
|Dự án The Empire ...|84.0|    NULL|       NULL|           NULL|             NULL|     4|    NULL|     NULL|Have certificate|           NULL|  8.6|
|Dự án The Crown -...|60.0|    NULL|       NULL|           NULL|             NULL|     5|    NULL|     NULL|            NULL|           NULL|  7.5|
|Dự án The Crown -...|90.0|     6.0|       13.0|     Đông - Bắc|       Đông - Bắc|     5|    NULL|     NULL|   Sale contract|           NULL|  8.9|
|Đường Nguyễn Văn ...|54.0|    NULL|        3.5|      Tây - Nam|        Tây - Nam|     2|       2|        3|Have

In [117]:
# xử lý giá trị null của cột numerical
# cột frontage and access_road 
cols = ['Frontage', "Access_Road", "Bedrooms", "Bathrooms"]
data = data.fillna({col : int (0) for col in cols}) 
# floor
data = data.fillna({'Floors':1})

In [118]:
data.groupBy('House_direction').count().show()

+---------------+-----+
|House_direction|count|
+---------------+-----+
|      Tây - Nam| 1152|
|      Tây - Bắc| 1125|
|           NULL|21238|
|            Tây|  791|
|     Đông - Nam| 1916|
|     Đông - Bắc| 1180|
|            Bắc|  832|
|            Nam| 1042|
|           Đông|  952|
+---------------+-----+



In [119]:
# xử lý giá trị null của cột String
mapping = {
    "Đông": "D",
    "Tây": "T",
    "Nam": "N",
    "Bắc": "B",
    "Đông - Bắc": "DB",
    "Đông - Nam": "DN",
    "Tây - Bắc": "TB",
    "Tây - Nam": "TN"
}
direction_cols = ["House_direction", "Balcony_direction"]

values = list(mapping.values())
n = len(values)

for col in direction_cols:
    data = data.withColumn(
        col,
        F.when(
            F.col(col).isNull(),
            F.element_at(F.array([F.lit(v) for v in values]), (F.rand()*n+1).cast("int"))
        ).otherwise(F.col(col))
    )

for col in direction_cols:
    for k, v in mapping.items():
        data = data.withColumn(
            col,
            F.when(F.col(col) == k, v).otherwise(F.col(col))
        )

data.groupBy('House_direction').count().show()



+---------------+-----+
|House_direction|count|
+---------------+-----+
|             TB| 3725|
|              T| 3502|
|              B| 3413|
|             DB| 3810|
|             DN| 4570|
|              D| 3634|
|             TN| 3806|
|              N| 3768|
+---------------+-----+



In [120]:
data = data.fillna({'Furniture_state': 'NO'})

In [121]:
data.groupBy('Legal_status').count().show()

+----------------+-----+
|    Legal_status|count|
+----------------+-----+
|   Sale contract|  949|
|Have certificate|24773|
|            NULL| 4506|
+----------------+-----+



In [122]:
data = data.fillna({'Legal_status' : 'Sale contract'})

In [123]:
null_count_df = data.select( [ F.count( F.when(F.col(col=c).isNull(), c)).alias(c) for c in data.columns] )
null_count_df.show()

+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+
|Address|Area|Frontage|Access_Road|House_direction|Balcony_direction|Floors|Bedrooms|Bathrooms|Legal_status|Furniture_state|Price|
+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+
|      0|   0|       0|          0|              0|                0|     0|       0|        0|           0|              0|    0|
+-------+----+--------+-----------+---------------+-----------------+------+--------+---------+------------+---------------+-----+



In [128]:
# trích xuất dữ liệu từ cốt Address
# Trích xuất vị trí
data = data.withColumn('parts_address', F.split(F.col('Address'), ','))

data = data.withColumn("Province_City",  F.trim(F.element_at(F.col("parts_address"), -1)))

data = data.withColumn(
    "Type",
    F.when(
        F.lower(F.trim(F.element_at(F.col("parts_address"), 1))).like("%dự án%"),
        F.lit("Chung cư")
    ).otherwise(F.lit("Nhà"))
)

data = data.drop("parts_address")

In [125]:
data.groupBy('Type').count().show()

+--------+-----+
|    Type|count|
+--------+-----+
|Chung cư| 3651|
|     Nhà|26577|
+--------+-----+



In [126]:
# chuẩn hóa dữ liệu

In [129]:
data.show()

+--------------------+-----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+-------------+--------+
|             Address| Area|Frontage|Access_Road|House_direction|Balcony_direction|Floors|Bedrooms|Bathrooms|    Legal_status|Furniture_state|Price|Province_City|    Type|
+--------------------+-----+--------+-----------+---------------+-----------------+------+--------+---------+----------------+---------------+-----+-------------+--------+
|Dự án The Empire ...| 84.0|     0.0|        0.0|             DB|               TB|     4|       0|        0|Have certificate|             NO|  8.6|     Hưng Yên|Chung cư|
|Dự án The Crown -...| 60.0|     0.0|        0.0|             TN|                B|     5|       0|        0|   Sale contract|             NO|  7.5|     Hưng Yên|Chung cư|
|Dự án The Crown -...| 90.0|     6.0|       13.0|             DB|               DB|     5|       0|        0|   Sale contract|             N